# Imports 

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib as plot 

# Data upload

In [2]:
shop_data = pd.read_csv('../raw_data/shop_sales_2021-07-01_2023-08-24.csv')

In [3]:
fb_data= pd.read_csv('../raw_data/fb_2023-08-24_AllTimeExcludingNL-Guido.csv')

In [4]:
tt_data = pd.read_excel('../raw_data/tt_Daily_01072022-30112022_DEAT.xlsx')

In [5]:
google_data=pd.read_excel('../raw_data/google_export.xlsx')

# Cleaning data

## droping columns

In [6]:
shop_data

,day,shipping_country,orders,total_sales
0,2021-07-01,Germany,10,414.85
1,2021-07-02,Austria,2,34.99
2,2021-07-02,Germany,4,194.96
3,2021-07-03,Germany,8,349.91
4,2021-07-03,Austria,2,109.98
...,...,...,...,...
1625,2023-08-23,Netherlands,19,720.36
1626,2023-08-23,Germany,121,4517.69
1627,2023-08-24,Netherlands,15,422.01
1628,2023-08-24,Austria,11,441.39


In [7]:
shop_data= shop_data.groupby('day', as_index=False)[['orders', 'total_sales']].sum()



In [8]:
shop_data

,day,orders,total_sales
0,2021-07-01,10,414.85
1,2021-07-02,6,229.95
2,2021-07-03,10,459.89
3,2021-07-04,17,508.36
4,2021-07-05,13,399.88
...,...,...,...
780,2023-08-20,309,14035.57
781,2023-08-21,372,15874.91
782,2023-08-22,171,6716.50
783,2023-08-23,149,5531.03


In [9]:
tt_data

,Date,Cost,Impression,Clicks (Destination),Reach,Total Complete Payment,Total Complete Payment Value,Currency
0,2022-07-01,320.00,122647,627,78947,2,49.98,EUR
1,2022-07-02,320.00,111337,624,72902,5,181.96,EUR
2,2022-07-03,368.29,137325,667,89738,6,244.93,EUR
3,2022-07-04,418.56,158848,885,99378,14,430.34,EUR
4,2022-07-05,420.00,178959,1011,113456,7,229.93,EUR
...,...,...,...,...,...,...,...,...
149,2022-11-27,110.00,26132,82,15680,1,49.99,EUR
150,2022-11-28,77.66,19912,73,12874,5,298.68,EUR
151,2022-11-29,49.75,19804,237,13834,2,80.48,EUR
152,2022-11-30,78.68,28702,351,20751,0,0.00,EUR


In [10]:
tt_data.drop(columns=['Currency', 'Total Complete Payment', 'Total Complete Payment Value', 'Reach'], inplace=True)

In [12]:
fb_data.drop(columns=['Reach', 'Reporting starts', 'Reporting ends'], inplace=True)

In [13]:
fb_data

,Day,Amount spent (EUR),Impressions,Link clicks
0,2023-08-24,1487.41,386594,839.0
1,2023-08-23,1315.81,337607,841.0
2,2023-08-22,1335.84,340325,839.0
3,2023-08-21,2210.01,595105,1284.0
4,2023-08-20,2148.53,555740,1366.0
...,...,...,...,...
854,2020-07-29,466.70,64296,460.0
855,2020-07-28,385.50,59499,386.0
856,2020-07-27,618.21,112787,770.0
857,2020-07-26,771.01,145720,1003.0


In [14]:
google_data.head()

,Tag,Währungscode,Kosten,Conversions,Impressionen,Klicks
0,2022-01-10,EUR,"1,91","6,00",65.000,29
1,2021-08-05,EUR,"2,61","0,00",62.000,24
2,2022-06-26,EUR,"5,15","10,00",142.000,43
3,2023-07-10,EUR,"215,64","49,75",8.996,398
4,2021-05-15,EUR,"0,01","0,00",5.000,1


In [15]:
google_data.drop(columns=['Währungscode', 'Conversions'], inplace=True)

## data info + type conversion + rename

In [16]:
google_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Tag           1126 non-null   datetime64[ns]
 1   Kosten        1126 non-null   object        
 2   Impressionen  1126 non-null   float64       
 3   Klicks        1126 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 35.3+ KB


In [17]:
google_data["Kosten"] = google_data.loc[:,["Kosten"]].replace({",":"."}, regex=True)

In [18]:
google_data

,Tag,Kosten,Impressionen,Klicks
0,2022-01-10,1.91,65.000,29
1,2021-08-05,2.61,62.000,24
2,2022-06-26,5.15,142.000,43
3,2023-07-10,215.64,8.996,398
4,2021-05-15,0.01,5.000,1
...,...,...,...,...
1121,2022-01-04,0.28,40.000,15
1122,2020-12-12,0.02,10.000,2
1123,2022-03-19,3.36,282.000,44
1124,2022-03-03,21.06,126.000,53


In [19]:
google_data['Kosten']=google_data['Kosten'].astype(float)

In [20]:
google_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Tag           1126 non-null   datetime64[ns]
 1   Kosten        1126 non-null   float64       
 2   Impressionen  1126 non-null   float64       
 3   Klicks        1126 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 35.3 KB


In [21]:
google_data['Impressionen']=google_data['Impressionen'].astype(int)

In [22]:
google_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Tag           1126 non-null   datetime64[ns]
 1   Kosten        1126 non-null   float64       
 2   Impressionen  1126 non-null   int64         
 3   Klicks        1126 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 35.3 KB


In [23]:
google_data=google_data.rename(columns={"Tag":"Day", 'Kosten':'google_costs', 'Impressionen':'google_impressions', 'Klicks':'google_clicks'})

In [24]:
google_data.head()

,Day,google_costs,google_impressions,google_clicks
0,2022-01-10,1.91,65,29
1,2021-08-05,2.61,62,24
2,2022-06-26,5.15,142,43
3,2023-07-10,215.64,8,398
4,2021-05-15,0.01,5,1


In [25]:
google_data.head()

,Day,google_costs,google_impressions,google_clicks
0,2022-01-10,1.91,65,29
1,2021-08-05,2.61,62,24
2,2022-06-26,5.15,142,43
3,2023-07-10,215.64,8,398
4,2021-05-15,0.01,5,1


In [26]:
tt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  154 non-null    object 
 1   Cost                  154 non-null    float64
 2   Impression            154 non-null    int64  
 3   Clicks (Destination)  154 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 4.9+ KB


In [27]:
tt_data

,Date,Cost,Impression,Clicks (Destination)
0,2022-07-01,320.00,122647,627
1,2022-07-02,320.00,111337,624
2,2022-07-03,368.29,137325,667
3,2022-07-04,418.56,158848,885
4,2022-07-05,420.00,178959,1011
...,...,...,...,...
149,2022-11-27,110.00,26132,82
150,2022-11-28,77.66,19912,73
151,2022-11-29,49.75,19804,237
152,2022-11-30,78.68,28702,351


In [28]:
tt_data.drop(tt_data.index[153], inplace=True) 

In [29]:
tt_data["Date"]=pd.to_datetime(tt_data["Date"])

In [30]:
tt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  153 non-null    datetime64[ns]
 1   Cost                  153 non-null    float64       
 2   Impression            153 non-null    int64         
 3   Clicks (Destination)  153 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 4.9 KB


In [32]:
tt_data=tt_data.rename(columns={'Date':'Day', 'cost':'tt_costs', 'Impression':'tt_impressions', 'Clicks (Destination)':'tt_clicks'})

In [33]:
tt_data=tt_data.rename(columns={'Cost':'tt_costs'})

In [34]:
tt_data

,Day,tt_costs,tt_impressions,tt_clicks
0,2022-07-01,320.00,122647,627
1,2022-07-02,320.00,111337,624
2,2022-07-03,368.29,137325,667
3,2022-07-04,418.56,158848,885
4,2022-07-05,420.00,178959,1011
...,...,...,...,...
148,2022-11-26,110.00,21425,88
149,2022-11-27,110.00,26132,82
150,2022-11-28,77.66,19912,73
151,2022-11-29,49.75,19804,237


In [35]:
fb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Day                 859 non-null    object 
 1   Amount spent (EUR)  859 non-null    float64
 2   Impressions         859 non-null    int64  
 3   Link clicks         817 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 27.0+ KB


In [36]:
fb_data["Day"]=pd.to_datetime(fb_data["Day"])

In [37]:
fb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 859 non-null    datetime64[ns]
 1   Amount spent (EUR)  859 non-null    float64       
 2   Impressions         859 non-null    int64         
 3   Link clicks         817 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 27.0 KB


In [38]:
fb_data.sort_values(by='Link clicks', ascending=False).tail(50)

,Day,Amount spent (EUR),Impressions,Link clicks
432,2022-06-16,19.76,4954,2.0
622,2021-12-04,21.58,3640,2.0
808,2020-11-05,1.23,270,2.0
593,2022-01-02,4.83,4692,2.0
635,2021-11-21,9.57,1228,1.0
431,2022-06-17,16.29,3629,1.0
430,2022-06-18,15.94,4069,1.0
429,2022-06-19,16.57,4213,1.0
427,2022-06-21,24.76,8091,NaN
428,2022-06-20,17.35,5925,NaN


In [39]:
fb_data['Impressions'].isnull().sum()

0

In [40]:
fb_data=fb_data.fillna(0)





In [41]:
fb_data['Link clicks'].isnull().sum()

0

In [42]:
fb_data['Link clicks']=fb_data['Link clicks'].astype(int)

In [43]:
fb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 859 non-null    datetime64[ns]
 1   Amount spent (EUR)  859 non-null    float64       
 2   Impressions         859 non-null    int64         
 3   Link clicks         859 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 27.0 KB


In [44]:
fb_data=fb_data.rename(columns={'Amount spent (EUR)':'fb_costs', 'Impressions':'fb_impressions', 'Link clicks':'fb_clicks'})

In [45]:
fb_data

,Day,fb_costs,fb_impressions,fb_clicks
0,2023-08-24,1487.41,386594,839
1,2023-08-23,1315.81,337607,841
2,2023-08-22,1335.84,340325,839
3,2023-08-21,2210.01,595105,1284
4,2023-08-20,2148.53,555740,1366
...,...,...,...,...
854,2020-07-29,466.70,64296,460
855,2020-07-28,385.50,59499,386
856,2020-07-27,618.21,112787,770
857,2020-07-26,771.01,145720,1003


In [46]:
shop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   day          785 non-null    object 
 1   orders       785 non-null    int64  
 2   total_sales  785 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 18.5+ KB


In [47]:
shop_data["day"]=pd.to_datetime(shop_data["day"])

In [48]:
shop_data=shop_data.rename(columns={'day':'Day'})

In [49]:
shop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Day          785 non-null    datetime64[ns]
 1   orders       785 non-null    int64         
 2   total_sales  785 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 18.5 KB


# Join tables

In [51]:
df=pd.merge(shop_data,fb_data , on="Day", how="outer").merge(google_data, on="Day").fillna(0)
df

,Day,orders,total_sales,fb_costs,fb_impressions,fb_clicks,google_costs,google_impressions,google_clicks
0,2021-07-01,10.0,414.85,257.01,36200.0,129.0,1.17,39,20
1,2021-07-02,6.0,229.95,250.30,33040.0,150.0,0.84,14,7
2,2021-07-03,10.0,459.89,248.59,33031.0,129.0,0.72,12,6
3,2021-07-04,17.0,508.36,258.05,37463.0,119.0,0.07,13,5
4,2021-07-05,13.0,399.88,257.47,35963.0,142.0,0.38,34,16
...,...,...,...,...,...,...,...,...,...
875,2020-07-29,0.0,0.00,466.70,64296.0,460.0,1.23,97,44
876,2020-07-28,0.0,0.00,385.50,59499.0,386.0,4.32,107,56
877,2020-07-27,0.0,0.00,618.21,112787.0,770.0,4.34,164,75
878,2020-07-26,0.0,0.00,771.01,145720.0,1003.0,0.35,60,27


In [52]:
shop_data.shape

(785, 3)

In [53]:
fb_data.shape

(859, 4)

In [54]:
google_data.shape

(1126, 4)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 880 non-null    datetime64[ns]
 1   orders              880 non-null    float64       
 2   total_sales         880 non-null    float64       
 3   fb_costs            880 non-null    float64       
 4   fb_impressions      880 non-null    float64       
 5   fb_clicks           880 non-null    float64       
 6   google_costs        880 non-null    float64       
 7   google_impressions  880 non-null    int64         
 8   google_clicks       880 non-null    int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 62.0 KB


In [59]:
df['fb_impressions']=df['fb_impressions'].astype(int)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 880 non-null    datetime64[ns]
 1   orders              880 non-null    float64       
 2   total_sales         880 non-null    float64       
 3   fb_costs            880 non-null    float64       
 4   fb_impressions      880 non-null    int64         
 5   fb_clicks           880 non-null    float64       
 6   google_costs        880 non-null    float64       
 7   google_impressions  880 non-null    int64         
 8   google_clicks       880 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 62.0 KB


In [61]:
df['fb_clicks']=df['fb_clicks'].astype(int)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880 entries, 0 to 879
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 880 non-null    datetime64[ns]
 1   orders              880 non-null    float64       
 2   total_sales         880 non-null    float64       
 3   fb_costs            880 non-null    float64       
 4   fb_impressions      880 non-null    int64         
 5   fb_clicks           880 non-null    int64         
 6   google_costs        880 non-null    float64       
 7   google_impressions  880 non-null    int64         
 8   google_clicks       880 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(4)
memory usage: 62.0 KB


In [65]:
df.to_csv('../raw_data/df.csv')